In [1]:
# 🚀 Google Colab Setup (Auto-detect)
import sys

# Check if running in Colab
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("🚀 Running in Google Colab")
    print("="*60)

    # Check GPU availability
    import torch
    if torch.cuda.is_available():
        print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
        print(f"   GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    else:
        print("⚠️  WARNING: No GPU detected!")
        print("   Please enable GPU:")
        print("   1. Go to Runtime → Change runtime type")
        print("   2. Select Hardware accelerator: GPU")
        print("   3. Click Save and restart")
        raise RuntimeError("GPU required for 4-bit quantization!")

    print("\n✅ Colab environment ready!")
    print("="*60)
else:
    print("💻 Running in local environment")
    print("="*60)

🚀 Running in Google Colab
✅ GPU available: Tesla T4
   GPU Memory: 15.83 GB

✅ Colab environment ready!


In [2]:
# Install necessary libraries
!pip install transformers datasets accelerate bitsandbytes torch pypdf huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.9/323.9 kB 16.6 MB/s eta 0:00:00


In [7]:
# Log in to your Hugging Face account
from huggingface_hub import notebook_login
notebook_login()

In [4]:
import torch
import json
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoModelForQuestionAnswering, TrainingArguments, Trainer
from datasets import load_dataset, Dataset
from pypdf import PdfReader

TEACHER_MODEL = "google/gemma-2-2b-it"  # Using smaller Gemma model
STUDENT_MODEL = "distilbert-base-uncased"

print(f"✅ Imports loaded successfully")
print(f"Teacher Model: {TEACHER_MODEL}")
print(f"Student Model: {STUDENT_MODEL}")

✅ Imports loaded successfully
Teacher Model: google/gemma-2-2b-it
Student Model: distilbert-base-uncased


In [5]:
# Step 1: PDF Upload and Text Extraction
from pypdf import PdfReader
import sys

def extract_text_from_pdf(pdf_path):
    """Extract all text from a PDF file"""
    reader = PdfReader(pdf_path)
    text = ""
    for page_num, page in enumerate(reader.pages):
        page_text = page.extract_text()
        if page_text:
            text += page_text + "\n"
    return text

# Check if running in Colab
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("📤 Upload your PDF file:")
    print("="*60)
    from google.colab import files
    uploaded = files.upload()
    pdf_path = list(uploaded.keys())[0]
    print(f"\n✅ File uploaded: {pdf_path}")
else:
    # Local environment - use file path
    pdf_path = "./book.pdf"  # Change this to your PDF file path
    print(f"📄 Using local file: {pdf_path}")

# Parse the PDF
print("\n" + "="*60)
print("Parsing PDF...")
full_text = extract_text_from_pdf(pdf_path)

print(f"✅ PDF parsed successfully")
print(f"Total characters extracted: {len(full_text):,}")
print(f"Total words (approx): {len(full_text.split()):,}")
print(f"\nPreview (first 500 chars):")
print("-"*60)
print(full_text[:500] + "...")
print("="*60)

📤 Upload your PDF file:


Saving The C programming Language.pdf to The C programming Language.pdf

✅ File uploaded: The C programming Language.pdf

Parsing PDF...
✅ PDF parsed successfully
Total characters extracted: 129,833
Total words (approx): 20,979

Preview (first 500 chars):
------------------------------------------------------------
The C programming Language
The C programming Language
By Brian W. Kernighan and Dennis M. Ritchie. 
Published by Prentice-Hall in 1988 
ISBN 0-13-110362-8 (paperback)
ISBN 0-13-110370-9 
Contents
●     Preface 
●     Preface to the first edition 
●     Introduction 
1.  Chapter 1: A Tutorial Introduction 
1.  Getting Started 
2.  Variables and Arithmetic Expressions 
3.  The for statement 
4.  Symbolic Constants 
5.  Character Input and Output 
1.  File Copying 
2.  Character Counting 
3.  Line ...


In [17]:
# Step 2: Tokenize and Chunk Text into 2k token chunks
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(TEACHER_MODEL)

CHUNK_SIZE = 2000  # 2k tokens per chunk
CHUNK_OVERLAP = 500  # Overlap to maintain context

def chunk_text_by_tokens(text, tokenizer, max_tokens, overlap):
    """
    Splits text into chunks of a maximum token size with overlap.
    """
    tokenized_input = tokenizer(text, return_tensors='pt',
                                add_special_tokens=False)['input_ids'][0]

    chunks = []
    for i in range(0, len(tokenized_input), max_tokens - overlap):
        chunk_tokens = tokenized_input[i : i + max_tokens]
        chunk_text = tokenizer.decode(chunk_tokens, skip_special_tokens=True)
        chunks.append(chunk_text)

        if i + max_tokens >= len(tokenized_input):
            break

    return chunks

text_chunks = chunk_text_by_tokens(full_text, tokenizer, CHUNK_SIZE, CHUNK_OVERLAP)

print(f"✅ Text chunked successfully")
print(f"Total chunks: {len(text_chunks)}")
print(f"Tokens per chunk: ~{CHUNK_SIZE}")
print(f"\nFirst chunk preview (first 300 chars):\n{text_chunks[0][:300]}...")

✅ Text chunked successfully
Total chunks: 23
Tokens per chunk: ~2000

First chunk preview (first 300 chars):
The C programming Language
The C programming Language
By Brian W. Kernighan and Dennis M. Ritchie. 
Published by Prentice-Hall in 1988 
ISBN 0-13-110362-8 (paperback)
ISBN 0-13-110370-9 
Contents
●     Preface 
●     Preface to the first edition 
●     Introduction 
1.  Chapter 1: A Tutorial Introdu...


In [19]:
# Step 3: Load Gemma Model in 4-bit Quantization (Teacher Model)
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# 4-bit quantization configuration to save memory
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Load model in 4-bit
    bnb_4bit_quant_type="nf4",  # Use normalized float 4-bit
    bnb_4bit_use_double_quant=True,  # Double quantization for better memory efficiency
    bnb_4bit_compute_dtype=torch.bfloat16  # Compute in bfloat16
)

print("Loading Gemma model in 4-bit quantization...")
gemma_tokenizer = AutoTokenizer.from_pretrained(TEACHER_MODEL)
gemma_model = AutoModelForCausalLM.from_pretrained(
    TEACHER_MODEL,
    device_map="auto",
    quantization_config=quant_config,
    torch_dtype=torch.bfloat16
)
gemma_model.eval()  # Set to evaluation mode

print(f"✅ Gemma model loaded successfully in 4-bit quantization")
print(f"Model device: {gemma_model.device}")

Loading Gemma model in 4-bit quantization...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Gemma model loaded successfully in 4-bit quantization
Model device: cuda:0


In [20]:
# Step 4: Create Prompt Template for JSON Generation
import json
import re

def create_qa_generation_prompt(text_chunk):
    """Create a prompt that instructs Gemma to generate QA pairs in JSON format"""
    prompt = f"""You are an expert educational content creator. Read the following text carefully and generate 5-8 high-quality question-answer pairs based on the content.

For each question-answer pair, include:
1. question: A clear, specific question about the content
2. answer: A detailed, accurate answer extracted from or based on the text
3. thinking_process: Your reasoning about why this question is important and how you arrived at the answer

Output ONLY a valid JSON array in this exact format:
[
  {{
    "question": "your question here",
    "answer": "your answer here",
    "thinking_process": "your reasoning here"
  }}
]

Text to analyze:
{text_chunk}

Generate the JSON array now:"""
    return prompt

print("✅ Prompt template created")

✅ Prompt template created


In [21]:
# Step 5: Generate QA pairs from each chunk using Gemma
def generate_qa_from_chunk(text_chunk, model, tokenizer):
    """Generate QA pairs from a text chunk using Gemma"""
    prompt = create_qa_generation_prompt(text_chunk)

    # Format as chat message
    messages = [{"role": "user", "content": prompt}]

    # Apply chat template
    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=True,
        return_tensors="pt",
        truncation=True,
        max_length=6000  # Leave room for generation (8192 - 2000 output)
    ).to(model.device)

    # Generate response
    with torch.inference_mode():
        output_ids = model.generate(
            input_ids=inputs,
            max_new_tokens=2000,  # Allow enough tokens for multiple QA pairs
            temperature=0.7,
            do_sample=True,
            top_p=0.9,
            eos_token_id=tokenizer.eos_token_id
        )

    # Decode output
    input_length = inputs.shape[1]
    decoded_output = tokenizer.decode(
        output_ids[0][input_length:],
        skip_special_tokens=True
    )

    return decoded_output.strip()

def parse_json_response(response_text):
    """Parse JSON from Gemma's response"""
    try:
        # Try to find JSON array in the response
        json_match = re.search(r'\[[\s\S]*\]', response_text)
        if json_match:
            json_str = json_match.group(0)
            qa_pairs = json.loads(json_str)
            return qa_pairs
        else:
            print("⚠️  Warning: No JSON array found in response")
            print(f"   Response preview: {response_text[:200]}...")
            return []
    except json.JSONDecodeError as e:
        print(f"⚠️  Error parsing JSON: {e}")
        print(f"   Response preview: {response_text[:200]}...")
        return []

print("✅ QA generation functions created")

✅ QA generation functions created


In [22]:
# Step 6: Process all chunks and collect QA pairs
all_qa_pairs = []

print(f"Processing {len(text_chunks)} chunks...")
for i, chunk in enumerate(text_chunks):
    print(f"\n{'='*50}")
    print(f"Processing chunk {i+1}/{len(text_chunks)}...")

    # Generate QA pairs for this chunk
    raw_response = generate_qa_from_chunk(chunk, gemma_model, gemma_tokenizer)

    # Parse JSON from response
    qa_pairs = parse_json_response(raw_response)

    # Add to collection
    all_qa_pairs.extend(qa_pairs)

    print(f"✅ Generated {len(qa_pairs)} QA pairs from chunk {i+1}")
    print(f"Total QA pairs so far: {len(all_qa_pairs)}")

    # Optional: Show first QA pair from this chunk
    if qa_pairs:
        print(f"\nSample QA from this chunk:")
        print(f"Q: {qa_pairs[0].get('question', 'N/A')[:100]}...")
        print(f"A: {qa_pairs[0].get('answer', 'N/A')[:100]}...")

print(f"\n{'='*50}")
print(f"✅ Processing complete!")
print(f"Total QA pairs generated: {len(all_qa_pairs)}")

Processing 23 chunks...

Processing chunk 1/23...
⚠️  Error parsing JSON: Expecting ',' delimiter: line 25 column 5 (char 3275)
   Response preview: [
  {
    "question": "What is the main purpose of the book 'The C Programming Language'?",
    "answer": "The book 'The C Programming Language' aims to provide a comprehensive and clear understanding...
✅ Generated 0 QA pairs from chunk 1
Total QA pairs so far: 0

Processing chunk 2/23...
✅ Generated 6 QA pairs from chunk 2
Total QA pairs so far: 6

Sample QA from this chunk:
Q: What led to the need for a more precise and contemporary definition of the C programming language?...
A: The computing world underwent a revolution since the publication of the first edition of The C Progr...

Processing chunk 3/23...
✅ Generated 6 QA pairs from chunk 3
Total QA pairs so far: 12

Sample QA from this chunk:
Q: What is C's relationship with the UNIX operating system?...
A: C has been closely associated with the UNIX operating system, where it was de

In [25]:
# Step 7: Save QA pairs to JSON file
output_file = "qa_dataset.json"

with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(all_qa_pairs, f, indent=2, ensure_ascii=False)

print(f"✅ Saved {len(all_qa_pairs)} QA pairs to '{output_file}'")

# Show statistics
print(f"\nDataset Statistics:")
print(f"- Total QA pairs: {len(all_qa_pairs)}")
if all_qa_pairs:
    print(f"\nFirst QA pair:")
    print(json.dumps(all_qa_pairs[0], indent=2))

# Clear GPU memory after generation
if torch.cuda.is_available():
    del gemma_model
    torch.cuda.empty_cache()
    print(f"\n✅ GPU memory cleared")

✅ Saved 94 QA pairs to 'qa_dataset.json'

Dataset Statistics:
- Total QA pairs: 94

First QA pair:
{
  "question": "What led to the need for a more precise and contemporary definition of the C programming language?",
  "answer": "The computing world underwent a revolution since the publication of the first edition of The C Programming Language in 1978. Big computers became much bigger, personal computers had capabilities rivaling mainframes of a decade ago.  The growing popularity of C, the changes in the language over the years, and the creation of compilers not involved in its design, combined to demonstrate a need for a more precise definition.",
  "thinking_process": "The text states that the computing world has changed significantly since the first edition.  The language has become more popular and more widely used, requiring a more precise and contemporary definition."
}

✅ GPU memory cleared


In [24]:
# 🔍 Debug: Check the QA dataset file
import os
import json

# Check if file exists and its size
if os.path.exists("qa_dataset.json"):
    file_size = os.path.getsize("qa_dataset.json")
    print(f"✅ File exists: qa_dataset.json")
    print(f"   File size: {file_size} bytes")

    if file_size == 0:
        print("\n❌ ERROR: File is empty!")
        print("   This means no QA pairs were generated successfully.")
        print("\n💡 Possible solutions:")
        print("   1. The chunks might be too large - try reducing CHUNK_SIZE to 2000")
        print("   2. Re-run Step 6 (Process all chunks) to regenerate QA pairs")
        print("   3. Check the output from Step 6 to see how many QA pairs were generated")
    else:
        print(f"\n📄 File content preview (first 500 chars):")
        with open("qa_dataset.json", 'r', encoding='utf-8') as f:
            content = f.read()
            print(content[:500])

        # Try to parse it
        try:
            qa_data = json.loads(content)
            print(f"\n✅ Valid JSON with {len(qa_data)} QA pairs")
        except json.JSONDecodeError as e:
            print(f"\n❌ Invalid JSON: {e}")
            print("   Re-run Step 6 to regenerate the file")
else:
    print("❌ File not found: qa_dataset.json")
    print("   You need to run Step 6 (Process all chunks) first!")

print(f"\n📊 Current QA pairs in memory: {len(all_qa_pairs) if 'all_qa_pairs' in dir() else 'Variable not found'}")

✅ File exists: qa_dataset.json
   File size: 0 bytes

❌ ERROR: File is empty!
   This means no QA pairs were generated successfully.

💡 Possible solutions:
   1. The chunks might be too large - try reducing CHUNK_SIZE to 2000
   2. Re-run Step 6 (Process all chunks) to regenerate QA pairs
   3. Check the output from Step 6 to see how many QA pairs were generated

📊 Current QA pairs in memory: 94


In [26]:
# Step 8: Format dataset for DistilBERT training
from datasets import Dataset
import pandas as pd

# Load QA pairs from file
with open("qa_dataset.json", 'r', encoding='utf-8') as f:
    qa_data = json.load(f)

# Convert to pandas DataFrame
df = pd.DataFrame(qa_data)

# For question answering, we need context.
# We'll use the answer + thinking_process as enriched context
df['context'] = df.apply(
    lambda row: f"Context: {row['answer']} Reasoning: {row['thinking_process']}",
    axis=1
)

# Create a proper QA dataset format
# For DistilBERT QA, we need: question, context, and answers (with answer_start)
qa_dataset_list = []
for idx, row in df.iterrows():
    qa_dataset_list.append({
        'id': str(idx),
        'question': row['question'],
        'context': row['context'],
        'answers': {
            'text': [row['answer']],
            'answer_start': [row['context'].find(row['answer'])]  # Find answer position in context
        }
    })

# Convert to HuggingFace Dataset
dataset = Dataset.from_list(qa_dataset_list)

# Split into train and validation
train_test = dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = train_test['train']
eval_dataset = train_test['test']

print(f"✅ Dataset formatted successfully")
print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(eval_dataset)}")
print(f"\nSample from training set:")
print(train_dataset[0])

✅ Dataset formatted successfully
Training samples: 84
Validation samples: 10

Sample from training set:
{'id': '33', 'question': 'What is the role of the `%ld` format specifier in the character counting program?', 'context': 'Context: The `%ld` format specifier tells `printf` that the argument is a long integer, which is necessary to display the count of characters accurately. This is because the character count might be larger than an integer can handle. Reasoning: The text explains that `%ld` is used to display long integers. This is important for accurately representing the count of characters. The text mentions that `%ld` is a format specifier for long integers, and it is used to display the count of characters.', 'answers': {'answer_start': [9], 'text': ['The `%ld` format specifier tells `printf` that the argument is a long integer, which is necessary to display the count of characters accurately. This is because the character count might be larger than an integer can handle.']}}


In [29]:
# Step 9: Prepare DistilBERT tokenization
from transformers import AutoTokenizer

distilbert_tokenizer = AutoTokenizer.from_pretrained(STUDENT_MODEL)

def preprocess_function(examples):
    """Tokenize questions and contexts for QA task"""
    questions = [q.strip() for q in examples["question"]]
    contexts = examples["context"]

    # Tokenize
    tokenized_examples = distilbert_tokenizer(
        questions,
        contexts,
        truncation="only_second",  # Only truncate context if needed
        max_length=384,
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length"
    )

    # Map back to original examples
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Initialize start and end positions
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(distilbert_tokenizer.cls_token_id)

        sequence_ids = tokenized_examples.sequence_ids(i)

        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]

        if len(answers["answer_start"]) == 0 or answers["answer_start"][0] == -1:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Find token start position
            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1

            # Find token end position
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            # Check if answer is in context
            if not (offsets[token_start_index][0] <= start_char and
                    offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Find exact token positions
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)

                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

# Tokenize datasets
tokenized_train = train_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=train_dataset.column_names
)

tokenized_eval = eval_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=eval_dataset.column_names
)

print(f"✅ Datasets tokenized successfully")
print(f"Tokenized training samples: {len(tokenized_train)}")
print(f"Tokenized validation samples: {len(tokenized_eval)}")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/84 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

✅ Datasets tokenized successfully
Tokenized training samples: 84
Tokenized validation samples: 10


In [30]:
# Step 10: Train DistilBERT on the generated QA dataset
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer, default_data_collator

# Load DistilBERT model for QA
distilbert_model = AutoModelForQuestionAnswering.from_pretrained(STUDENT_MODEL)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./distilbert-qa-finetuned",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
    logging_steps=10,
    save_strategy="epoch",
    load_best_model_at_end=True,
    fp16=torch.cuda.is_available(),  # Use mixed precision if GPU available
)

# Create Trainer
trainer = Trainer(
    model=distilbert_model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    tokenizer=distilbert_tokenizer,
    data_collator=default_data_collator,
)

# Train the model
print("Starting DistilBERT training...")
print(f"Training samples: {len(tokenized_train)}")
print(f"Validation samples: {len(tokenized_eval)}")
print(f"Epochs: {training_args.num_train_epochs}")

trainer.train()

print("\n✅ Training complete!")

# Save the final model
trainer.save_model("./distilbert-qa-final")
distilbert_tokenizer.save_pretrained("./distilbert-qa-final")

print(f"✅ Model saved to './distilbert-qa-final'")

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-375456673.py:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Starting DistilBERT training...
Training samples: 84
Validation samples: 10
Epochs: 3


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: karthikeyanrv (government-college-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,5.386100,4.802637
2,4.230600,3.908203
3,3.486100,3.557275



✅ Training complete!
✅ Model saved to './distilbert-qa-final'


In [31]:
# Step 11: Test the trained model (Optional)
from transformers import pipeline

# Load the trained model
qa_pipeline = pipeline(
    "question-answering",
    model="./distilbert-qa-final",
    tokenizer="./distilbert-qa-final"
)

# Test with a sample question
test_context = "DistilBERT is a smaller, faster version of BERT. It retains 97% of BERT's performance while being 40% smaller and 60% faster."
test_question = "What is DistilBERT?"

result = qa_pipeline(question=test_question, context=test_context)

print("Testing the trained model:")
print(f"Question: {test_question}")
print(f"Context: {test_context}")
print(f"\nAnswer: {result['answer']}")
print(f"Confidence: {result['score']:.4f}")

print("\n✅ Pipeline complete! Your DistilBERT model is trained and ready to use.")

Device set to use cuda:0


Testing the trained model:
Question: What is DistilBERT?
Context: DistilBERT is a smaller, faster version of BERT. It retains 97% of BERT's performance while being 40% smaller and 60% faster.

Answer: DistilBERT is a smaller, faster version of BERT.
Confidence: 0.0272

✅ Pipeline complete! Your DistilBERT model is trained and ready to use.


In [32]:
# 💾 Download Results (Colab Only)
import sys

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("📦 Preparing files for download...")
    print("="*60)

    # Zip the trained model
    import shutil
    shutil.make_archive('distilbert-qa-final', 'zip', '.', 'distilbert-qa-final')
    print("✅ Model zipped: distilbert-qa-final.zip")

    # Download files
    from google.colab import files

    print("\n📥 Downloading files...")
    print("1. Downloading trained model (distilbert-qa-final.zip)...")
    files.download('distilbert-qa-final.zip')

    print("2. Downloading QA dataset (qa_dataset.json)...")
    files.download('qa_dataset.json')

    print("\n" + "="*60)
    print("✅ Downloads complete!")
    print("\nYou can also save to Google Drive:")
    print("   from google.colab import drive")
    print("   drive.mount('/content/drive')")
    print("   !cp -r ./distilbert-qa-final /content/drive/MyDrive/")
    print("   !cp qa_dataset.json /content/drive/MyDrive/")
    print("="*60)
else:
    print("💻 Running locally - files saved to disk:")
    print("   - ./distilbert-qa-final/ (trained model)")
    print("   - ./qa_dataset.json (QA pairs)")
    print("   - ./distilbert-qa-finetuned/ (training checkpoints)")

📦 Preparing files for download...
✅ Model zipped: distilbert-qa-final.zip

📥 Downloading files...
1. Downloading trained model (distilbert-qa-final.zip)...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

2. Downloading QA dataset (qa_dataset.json)...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ Downloads complete!

You can also save to Google Drive:
   from google.colab import drive
   drive.mount('/content/drive')
   !cp -r ./distilbert-qa-final /content/drive/MyDrive/
   !cp qa_dataset.json /content/drive/MyDrive/


In [33]:
# 🎯 How to Use Your Trained Model - Interactive Q&A
from transformers import pipeline

# Load your trained model
qa_model = pipeline(
    "question-answering",
    model="./distilbert-qa-final",
    tokenizer="./distilbert-qa-final"
)

print("="*60)
print("🎓 C Programming Q&A System")
print("="*60)
print("\nYour model can answer questions about C programming concepts")
print("that were learned from 'The C Programming Language' book.\n")

# Example questions you can ask
example_questions = [
    {
        "question": "What is the main purpose of the C programming language?",
        "context": "C is a general-purpose programming language designed for system programming. It provides low-level access to memory and efficient execution, making it ideal for operating systems and embedded systems."
    },
    {
        "question": "What is a pointer in C?",
        "context": "A pointer is a variable that stores the memory address of another variable. Pointers are declared using the asterisk (*) symbol and allow direct memory manipulation."
    },
    {
        "question": "How do you declare an array in C?",
        "context": "Arrays in C are declared by specifying the data type, array name, and size in square brackets. For example: int numbers[10] declares an array of 10 integers."
    }
]

# Test with example questions
print("📝 Example Questions & Answers:\n")
for i, example in enumerate(example_questions[:3], 1):
    result = qa_model(question=example["question"], context=example["context"])
    print(f"{i}. Q: {example['question']}")
    print(f"   A: {result['answer']}")
    print(f"   Confidence: {result['score']:.4f}\n")

print("="*60)
print("💡 Usage Tips:")
print("="*60)
print("1. Load the model: qa_model = pipeline('question-answering', model='./distilbert-qa-final')")
print("2. Ask questions: result = qa_model(question='your question', context='relevant text')")
print("3. Get answer: result['answer'] and confidence: result['score']")
print("\n⚠️  Important: You MUST provide context (relevant text passage)")
print("   The model extracts answers FROM the context you provide.")
print("="*60)

Device set to use cuda:0


🎓 C Programming Q&A System

Your model can answer questions about C programming concepts
that were learned from 'The C Programming Language' book.

📝 Example Questions & Answers:

1. Q: What is the main purpose of the C programming language?
   A: It provides low-level access to memory and efficient execution,
   Confidence: 0.0171

2. Q: What is a pointer in C?
   A: the asterisk (*) symbol and allow direct memory manipulation.
   Confidence: 0.0242

3. Q: How do you declare an array in C?
   A: the data type, array name, and size in square brackets.
   Confidence: 0.0269

💡 Usage Tips:
1. Load the model: qa_model = pipeline('question-answering', model='./distilbert-qa-final')
2. Ask questions: result = qa_model(question='your question', context='relevant text')
3. Get answer: result['answer'] and confidence: result['score']

⚠️  Important: You MUST provide context (relevant text passage)
   The model extracts answers FROM the context you provide.
